# Application of Radial Equilibrium Equation for a Rotor
## Adaptation of the code RE-DES by Lewis (Turbomachines Performance Analysis)

The radial equilibrium equation can be reordened and written as

$$\boxed{
\frac{\text{d}}{\text{d}r} c_x(r)^2 = 2\left( \omega - \frac{c_\theta(r)}{r}\right) 
\frac{\text{d} (rc_\theta(r))}{\text{d}r}
}  \tag{1}
$$

that gives the solution 
$$
c_x(r) = \sqrt{f(r) + k} \tag{2}
$$
where
$$
f(r) = \int_{r_h}^r \left( \omega - \frac{c_\theta(r)}{r}\right)\text{d} (rc_\theta(r)) \tag{3}
$$

The aim is, given all the data: $Q$, $\omega$, $r_h$, $r_t$ and the function 
$c_\theta(r)$, compute $c_x(r)$ and the angle $\beta_2(r)$ that fullfils the 
required flowrate.

The data ara given in the next cells 

In [149]:
import numpy as np
from scipy.interpolate import CubicSpline
from scipy.integrate import cumulative_trapezoid, trapezoid

In [165]:
Dh = 0.3   #m
rh = Dh/2  #m
Dt = 1.250 #m
rt = Dt/2  #m
rdata = np.linspace(rh,rt,10)
Qdata = 1.41e5 #m^3/h
Qdata = Qdata/3600 # m³/s
omega = 1450  #rpm
omega = omega*np.pi/30  #rad/s
A = 100
B = 7.776
#cthetadata = A*rdata                 ## forced vortex (solid body swirl)
#cthetadata = A*np.ones(rdata.size)   ## constant ctheta
#cthetadata = B/rdata                 ## free-vortex
cthetadata = A*(rdata-rms) + B/rdata  ## Arbitrary vortex
#cthetadata = 2*omega/3*rdata-9.08/np.sqrt(rdata) ## Another arbitrary vortex

The hub to tip ratio and the rms radius, $r_\text{rms} = \sqrt{\frac{r_h^2+r_t^2}{2}}$ are  computed. Also, $c_{\theta,\text{rms}}=c_\theta(r=r_\text{rms})$ is calculated as an interpolation of given data, and  $c_x$ in $r_\text{rms}$ is as well estimated, assuming that it is the average velocity given by the flow rate, $\overline{c_x}$.
For numerical computations, the $r$ domain is divided in $m$ intervals.

In [166]:
h = rh/rt
rrms = np.sqrt(0.5*(rh*rh+rt*rt))
ctrms = CubicSpline(rdata,cthetadata)(rrms)
cxm = Qdata/(np.pi*(rt*rt-rh*rh))
m = 601   # number of interpolation points
r = np.linspace(rh,rt,m)
print("Hub to tip ratio = {:0.4f}".format(h))
print("rms = {:.4f} m".format(rrms))
print("ctheta_rms = {:.4f} m/s".format(ctrms))
print("cx_rms = {:.4f} m/s".format(cxm))

Hub to tip ratio = 0.2400
rms = 0.4545 m
ctheta_rms = 17.1090 m/s
cx_rms = 33.8666 m/s


And now, the function $f(r)$ is computed by numerical integration (Eq. (3))

In [167]:
ctheta = CubicSpline(rdata,cthetadata)(r)
f = cumulative_trapezoid(omega-np.divide(ctheta,r),
                         np.multiply(r,ctheta),initial=0)

### First approximation

The first aproximation of the value of $k$ is with the assumption that 
$c_{x,\text{rms}} = \overline{c_x}$

In [178]:
frms = CubicSpline(r,f)(rms)
k = cxm*cxm-frms
print("First approximation of k: \n k = {:.4f} m²/s²".format(k))

First approximation of k: 
 k = 578.4354 m²/s²


The obtained solution is printed

In [179]:
def printSolution(k):
    cx = np.sqrt(k + f)
    Q = 2*np.pi*trapezoid(np.multiply(r,cx),r)
    print("{:^15}{:^10}{:^10}{:^10}{:^10}{:^10}{:^10}"
          .format("Radius","ctheta","cx","alpha2","beta1","beta2","beta_m"))
    print("\u2500"*80)
    for i in range(rdata.size):
        cxans = CubicSpline(r,cx)(rdata[i])
        alpha2 = np.rad2deg(np.arctan(cthetadata[i]/cxans))
        beta2 = np.rad2deg(np.arctan((omega*rdata[i]-cthetadata[i])/cxans))
        beta1 = np.rad2deg(np.arctan(omega*rdata[i]/cxans))
        beta_m = 0.5*(beta1+beta2)
        print("{:^15.4f}{:^10.2f}{:^10.2f}{:^10.2f}{:^10.2f}{:^10.2f}{:^10.2f}".
              format(rdata[i],cthetadata[i],cxans,alpha2,beta1,beta2,beta_m))
    print("Q = {:.3f} m^3/s".format(Q))
    errorQ = np.abs(Qdata-Q)/Qdata
    print("error = {:.1e} %".format(errorQ))
    
printSolution(k)

    Radius       ctheta      cx      alpha2    beta1     beta2     beta_m  
────────────────────────────────────────────────────────────────────────────────
    0.1500       21.39     24.05     41.65     43.44      3.30     23.37   
    0.2028       13.18     23.48     29.30     52.67     36.88     44.78   
    0.2556       10.53     23.55     24.10     58.75     50.21     54.48   
    0.3083       10.60     24.92     23.05     61.97     55.47     58.72   
    0.3611       12.20     27.48     23.93     63.38     57.20     60.29   
    0.4139       14.73     30.88     25.50     63.84     57.31     60.57   
    0.4667       17.88     34.81     27.19     63.84     56.69     60.27   
    0.5194       21.47     39.06     28.79     63.65     55.77     59.71   
    0.5722       25.36     43.50     30.24     63.41     54.74     59.07   
    0.6250       29.49     48.04     31.55     63.15     53.70     58.43   
Q = 39.482 m^3/s
error = 8.1e-03 %


### More precise computation


In [180]:
from scipy.optimize import brentq

In [181]:
def QFunction(k):
    cx = np.sqrt(k + f)
    Q_temptative = 2*np.pi*trapezoid(np.multiply(r,cx),r)
    return Qdata-Q_temptative

k = brentq(QFunction,0.5*k,1.5*k)
print("k = {:.4f} m²/s²".format(k))

k = 560.9236 m²/s²


In [182]:
printSolution(k)

    Radius       ctheta      cx      alpha2    beta1     beta2     beta_m  
────────────────────────────────────────────────────────────────────────────────
    0.1500       21.39     23.68     42.09     43.88      3.35     23.61   
    0.2028       13.18     23.10     29.70     53.12     37.33     45.22   
    0.2556       10.53     23.17     24.45     59.16     50.66     54.91   
    0.3083       10.60     24.57     23.35     62.31     55.85     59.08   
    0.3611       12.20     27.16     24.18     63.65     57.51     60.58   
    0.4139       14.73     30.59     25.71     64.05     57.55     60.80   
    0.4667       17.88     34.56     27.36     64.00     56.89     60.44   
    0.5194       21.47     38.84     28.93     63.78     55.92     59.85   
    0.5722       25.36     43.30     30.36     63.51     54.86     59.19   
    0.6250       29.49     47.86     31.64     63.24     53.81     58.52   
Q = 39.167 m^3/s
error = 1.8e-16 %
